## Prepare training/test data

## Import Library

### Installations

In [ ]:
pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 2.8 MB/s eta 0:00:00


In [ ]:
pip install transformers

In [ ]:
pip install torchtext

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.0 MB/s eta 0:00:00


In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

### Imports

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from catboost import CatBoostClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay
from tqdm import tqdm

import seaborn as sns

import random
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import classification_report, f1_score

## Load data

In [ ]:
df = pd.read_csv('final_bin_df_politicts_v1.csv', index_col=0)
df.label = df.label.astype(int)

In [ ]:
df_train, df_test = train_test_split(df,
                                    random_state=42,
                                    test_size=0.2,
                                    stratify=df.label
                                    )
df_test, df_val = train_test_split(df_test,
                                    random_state=42,
                                    test_size=0.2,
                                    stratify=df_test.label
                                    )
train_corpus = df_train.text.values
test_corpus = df_test.text.values
val_corpus = df_val.text.values

y_train = df_train.label
y_test = df_test.label
y_val = df_val.label

vectorizer = TfidfVectorizer(ngram_range=(2,4), analyzer='char_wb', max_df=0.8, min_df=10)
vectorizer.fit(df.text)
X_train = vectorizer.transform(train_corpus)
X_test = vectorizer.transform(test_corpus)
X_val = vectorizer.transform(val_corpus)

## RandomForestClassifier

In [ ]:
# cat_features = list(range(0, X.shape[1]))
# print(cat_features)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
y_pred = rfc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1138
           1       0.87      0.83      0.85      1138

    accuracy                           0.85      2276
   macro avg       0.85      0.85      0.85      2276
weighted avg       0.85      0.85      0.85      2276



## CatBoost

In [ ]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=15,
    learning_rate=0.001,
    loss_function='CrossEntropy'
)

clf.fit(X_train, y_train,
        eval_set=(X_val, y_val),
        verbose=False
)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.29      0.43      1138
           1       0.57      0.92      0.70      1138

    accuracy                           0.61      2276
   macro avg       0.68      0.61      0.56      2276
weighted avg       0.68      0.61      0.56      2276



Дальше следует попробовать нейронные сети

## Promt Learning Models

Installing openprompt library: open source solution for handling prompting methods

In [ ]:
pip install openprompt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99


Defining parameters

In [ ]:
# VERBALIZER_OPTIONS = (
#     "manual",
#     "soft"
# )
VERBALIZER_MODE = "soft"

### Define a task

Reading marked data

In [ ]:
input_file = '/content/any_nn_cleaned_bin_df_politics.csv'

df = pd.read_csv(input_file)
df.label = df.label.astype(int)
df.head()

,Unnamed: 0,text,label,assessment
0,0,"одна из тех не многих игр, по прохождению кото...",1,1.0
1,1,"узнал , про эту игру из ленты новостей. я как ...",1,0.0
2,2,прошел я tales of bayun от российской студии f...,1,0.0
3,3,"купил эту игру из подборки, в порыве поддержат...",1,0.0
4,4,"по совету проверенного товарища, третьего дня ...",1,1.0


Definition of classes in classification problem

In [ ]:
classes = [
    'политический',
    'нейтральный'
]

Creating dataset for the model

In [ ]:
pip install transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.0.dev0
    Uninstalling transformers-4.42.0.dev0:
      Successfully uninstalled transformers-4.42.0.dev0


In [ ]:
def train_val_test_split(data, proportions=None, shuffle=True):
    if proportions is None:
        proportions = [0.8, 0.2, 0]
    else:
        assert sum(proportions) == 1
    train_size = int(proportions[0] * len(data))
    val_size = int(proportions[1] * len(data))

    data_c = data.copy()

    if shuffle:
        np.random.shuffle(data_c)

    return {
        'train': data_c[:train_size],
        'validation': data_c[train_size:train_size+val_size],
        'test': data_c[train_size+val_size:]
    }

In [ ]:
from openprompt.data_utils import InputExample
full_input_examples = []
for index, row in df.iterrows():
    full_input_examples.append(
        InputExample(
            guid=index,
            text_a=row['text'],
            label=row['label']
        )
    )

In [ ]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers

Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gw5pdvws
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gw5pdvws
  Resolved https://github.com/huggingface/transformers to commit 4b822560a1dfd5d63c985ecf9a3c0aae0a4feeee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9231578 sha256=4875b7034edaf57a040f326efae58fc8806a0b9e91dba0ac43355094a1bb5d8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-1jhg1vyl/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [ ]:
dataset = train_val_test_split(full_input_examples)

In [ ]:
print(dataset['train'][0])

{
  "guid": 1955,
  "label": 1,
  "meta": {},
  "text_a": "\u043e\u0442\u0432\u0440\u0430\u0442\u043d\u0430\u044f \u043f\u043e\u043b\u0438\u0442\u043e\u0442\u0430. \u043f\u0435\u0448\u043a\u0438 \u0447\u0435\u0448\u0443\u0442 \u0447\u0441\u0432.",
  "text_b": "",
  "tgt_text": null
}



### Obtain a PLM

In [ ]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm(
    "bert",
    "DeepPavlov/rubert-base-cased-conversational"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Define a Template

In [ ]:
from openprompt.prompts import ManualTemplate
template_text = '{"placeholder":"text_a"}. Этот текст {"mask"}'
my_template = ManualTemplate(
    text=template_text,
    tokenizer=tokenizer,
)

In [ ]:
wrapped_example = my_template.wrap_one_example(
    dataset['train'][0]
)

###  Define a wrapped tokenizer

In [ ]:
wrapped_tokenizer = WrapperClass(
    max_seq_length=128,
    decoder_max_length=3,
    tokenizer=tokenizer,
    truncate_method='head'
)

In [ ]:
tokenized_example = wrapped_tokenizer.tokenize_one_example(
    wrapped_example,
    teacher_forcing=False
)

print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
#print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))

{'input_ids': [101, 9630, 5214, 29786, 132, 1726, 1635, 100158, 362, 22676, 132, 132, 4520, 6231, 103, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'loss_ids': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
model_inputs = {}

for split in ['train', 'validation', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_tokenizer.tokenize_one_example(
            my_template.wrap_one_example(sample),
            teacher_forcing=False
        )
        model_inputs[split].append(tokenized_example)

### Define dataloader

In [ ]:
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(
    dataset=dataset['train'],
    template=my_template,
    tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=256,
    decoder_max_length=3,
    batch_size=16,
    shuffle=True,
    teacher_forcing=False,
    predict_eos_token=False,
    truncate_method='head'
)

tokenizing: 9423it [00:21, 436.34it/s]


### Define a Verbalizer

In [ ]:
from openprompt.prompts import ManualVerbalizer, SoftVerbalizer
import torch

if VERBALIZER_MODE == "manual":
    my_verbalizer = ManualVerbalizer(
        tokenizer,
        classes=classes,
        num_classes=2,
        label_words = {
            "политический": ["политический"],
            "нейтральный": ["нейтральный"]
        },
    )
elif VERBALIZER_MODE == "soft":
    my_verbalizer = SoftVerbalizer(
        tokenizer,
        plm,
        num_classes=2,
        label_words=["политический", "нейтральный"],
    )
else:
    raise NotImplementedError(
        f"VERBALIZER_MODE has incorrect value `{VERBALIZER_MODE}`"
    )

### Construct a PromptModel

In [ ]:
from openprompt import PromptForClassification

device = 'cuda' if torch.cuda.is_available() else 'cpu'
prompt_model = PromptForClassification(
    plm=plm,
    template=my_template,
    verbalizer=my_verbalizer,
    freeze_plm=False if VERBALIZER_MODE == "soft" else False
)
prompt_model = prompt_model.to(device)

### Training

#### Prepare loss, optimizers

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

loss_func = torch.nn.CrossEntropyLoss().to(device)
no_decay = ['bias', 'LayerNorm.weight']
# it is always a good practice to set no decay to biase and LayerNorm parameters

optimizer_grouped_parameters = [
    {
        'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)],
        'weight_decay': 0.01
    },
    {
        'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)],
        'weight_decay': 0.0
    }
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  weight_decay=0.0,


#### Training process

In [ ]:
for epoch in range(4):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        inputs = inputs.to(device)
        logits = prompt_model(inputs)
        labels = inputs['label']
        #print(logits, labels)
        loss = loss_func(logits, labels)
        #loss.requires_grad = True
        loss.backward()

        tot_loss += loss.item()

        optimizer.step()
        optimizer.zero_grad()

        if step % 10 == 0:
            print(f'Epoch {epoch}, average loss: {tot_loss / (step + 1)}', flush=True)

Epoch 0, average loss: 3.1831960678100586
Epoch 0, average loss: 2.494244732640006
Epoch 0, average loss: 1.6497181568826949
Epoch 0, average loss: 1.3306926988786267
Epoch 0, average loss: 1.1745326926068562
Epoch 0, average loss: 1.065323787577012
Epoch 0, average loss: 0.9884909259491279
Epoch 0, average loss: 0.9366954720356095
Epoch 0, average loss: 0.8952316759545126
Epoch 0, average loss: 0.8621168909492073
Epoch 0, average loss: 0.8414341417869718
Epoch 0, average loss: 0.82037138858357
Epoch 0, average loss: 0.7934330288043692
Epoch 0, average loss: 0.7780770561622299
Epoch 0, average loss: 0.754983459804075
Epoch 0, average loss: 0.7462142494735339
Epoch 0, average loss: 0.735659240565685
Epoch 0, average loss: 0.7295783008748328
Epoch 0, average loss: 0.7293273490436828
Epoch 0, average loss: 0.7238175453939987


KeyboardInterrupt: 

### Model evaluation

Prepare test dataloader

In [ ]:
test_dataloader = PromptDataLoader(
    dataset=dataset['validation'],
    template=my_template,
    tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=256,
    decoder_max_length=3,
    batch_size=4,
    shuffle=False,
    teacher_forcing=False,
    predict_eos_token=False,
    truncate_method='head'
)

Evaluation process

In [ ]:
all_preds = []
all_labels = []

for step, inputs in enumerate(test_dataloader):
    inputs = inputs.to(device)
    logits = prompt_model(inputs)
    labels = inputs['label']
    #print(labels)
    all_labels.extend(labels.cpu().tolist())
    all_preds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(all_labels, all_preds, digits=4))